# 211003 xxx

In [1]:
from pathlib import Path
import json

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import h5py as h5

In [25]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## Setup

In [3]:
infiles = dict(
    fastq_data=Path('data-intermediate/211003-find-kmers/fastq-data.h5'),
)

In [ ]:
data_intermediate = Path('data-intermediate') / NBNAME
data_intermediate.mkdir(exist_ok=True)

outfiles = dict(
    assembled_sigs=data_intermediate / 'assembled-signatures.h5',
    fastq_data=data_intermediate / 'fastq-data.h5',
)

## Load data

In [6]:
agg_methods = ['min', 'phredsum']

In [57]:
data = dict()

with h5.File(infiles['fastq_data']) as f:
    
    for key, group in f.items():
        item = data[key] = dict()

        for m in agg_methods:
            item[m] = {
                k: group[f'{m}_{k}'][:]
                for k in ['counts', 'in_assembly', 'indices']
            }

In [58]:
genome_names = list(data)

In [73]:
index = pd.MultiIndex.from_product([genome_names, agg_methods], names=['genome', 'agg'])
index_pn = pd.MultiIndex.from_product([genome_names, agg_methods, [True, False]], names=['genome', 'agg', 'positive'])

## ?

In [72]:
_rows = []

for genome, agg in index:
    d = data[genome][agg]
    
    ispos = d['in_assembly']
    
    d['npos'] = np.sum(ispos)
    d['nneg'] = np.sum(~ispos)
    
    d['counts_pos'] = d['counts'][ispos, :]
    d['counts_neg'] = d['counts'][~ispos, :]
    
    d['ccounts'] = np.cumsum(d['counts'][:, ::-1], axis=1)[:, ::-1]
    d['ccounts_pos'] = d['ccounts'][ispos, :]
    d['ccounts_neg'] = d['ccounts'][~ispos, :]

In [68]:
_rows = []

for genome, agg in index:
    d = data[genome][agg]
    
    _rows.append(dict(
        max_count=np.max(np.sum(d[''], axis=0))
    ))
    
df = pd.DataFrame.from_records(_rows, index=index)

In [75]:
_rows = []

for genome, agg in index:
    d = data[genome][agg]
    
    _rows.append(dict(
        count=d['nneg'],
        max_kmer_ccount=np.max(np.sum(d['ccounts_neg'], axis=0))
    ))
    _rows.append(dict(
        count=d['npos'],
        max_kmer_ccount=np.max(np.sum(d['ccounts_pos'], axis=0))
    ))
    
df2 = pd.DataFrame.from_records(_rows, index=index_pn)

In [76]:
df2

count  max_kmer_ccount
genome                agg      positive                        
16AC1611137-CAP_S12   min      True       9825            13966
                               False      7341           401002
                      phredsum True       9825            13966
                               False      7341           401002
16AC1611138-CAP_S13   min      True       9001            12306
...                                        ...              ...
20AC0000922-C2ANO2_S9 phredsum False     12276           355060
20AC0000923-C_S11     min      True       7412            30833
                               False      5787           459779
                      phredsum True       7412            30833
                               False      5787           459779

[320 rows x 2 columns]